In [1]:
import numpy as np
import tensorflow as tf 
import tflearn
from collections import deque
import random
import pdb
from replay_memory import ReplayMemory
from AC_networks import ActorNet, CriticNet
#from test_game import test_env
import os

In [2]:
class input_struct:
    def __init__(self, net):
        self.net = net


In [3]:
import gym
game = gym.make('LunarLander-v2')
#game = gym.make('CartPole-v0')
state_dim = game.observation_space.shape[0]
action_dim = game.action_space.n

[2016-12-28 20:34:14,225] Making new env: LunarLander-v2


In [19]:
# info = graph, dim_s, dim_a, hid_layers, learning_rate, tau
save_dir = './Results/net/'
MAX_EPISODE = 20000
MAX_STEP = 1000
MINIBATCH_SIZE = 16
GAMMA = 0.99
LEARNING_RATE = 1e-2 #5e-5 lr/2
TAU = 0
ENTROPY_BETA = 0
buff = ReplayMemory()
info = input_struct('actor')
try:
    actor
except:
    pass
else:
    actor.reset_net()
try:
    critic
except:
    pass
else:
    critic.reset_net()  
    
graph = tf.Graph()
with graph.as_default():
    info.graph = graph
    info.dim_s = state_dim
    info.dim_a = action_dim
    info.act_hid_layers = np.array([16])
    info.crit_hid_layers = np.array([16])
    info.learning_rate = LEARNING_RATE
    info.tau = TAU
    info.ent_beta = ENTROPY_BETA
    actor = ActorNet(info)
    critic = CriticNet(info, actor.get_num_trainable_vars())

In [21]:
#game = test_env()
print_ep = 100
save_ep = 1000
lr_decay_ep = 100
decay_rate = 1
lr = LEARNING_RATE
buff.clear()
last_ep = 0
start_update = 0
with tf.Session(graph=graph) as sess:
    actor.set_up(sess)
    critic.set_up(sess)
    sess.run(tf.initialize_all_variables())
    
    ep_reward = np.zeros(MAX_EPISODE)
    ep_success = np.zeros(MAX_EPISODE)
    ep_crash = np.zeros(MAX_EPISODE)
    avg_var = 0
    
    for ep in xrange(MAX_EPISODE):
        s = game.reset()
        var = 0
        for st in xrange(MAX_STEP):
            A = actor.take_action(s.reshape(-1,info.dim_s))
            a = np.argmax(A)
            s2, r, terminal, report = game.step(a)

            buff.add(np.reshape(s, (actor.dim_s,)).astype(np.float32), np.reshape(A, (actor.dim_a,)).astype(np.float32),\
                     np.array(r).astype(np.float32), np.reshape(s2, (actor.dim_s,)).astype(np.float32),\
                     np.reshape(terminal, (1,)))

            ep_reward[ep] += (1.*r)
            var += np.sum(actor.give_policy(s.reshape(-1,info.dim_s))**2)
            s = s2

            if terminal:
                var = var/st
                avg_var += (var/print_ep)
                break
                
        if ((ep % MINIBATCH_SIZE) == (MINIBATCH_SIZE -1)):
            (s_batch, a_batch, r_batch, s2_batch, t_batch) = \
                buff.sample_batch(MINIBATCH_SIZE)
            s_batch = np.array(s_batch).reshape(-1, info.dim_s)
            s2_batch = np.array(s2_batch).reshape(-1, info.dim_s)
            a_batch = np.array(a_batch).reshape(-1, info.dim_a)
            r_batch = np.array(r_batch).reshape(-1, 1)
            t_batch = np.array(t_batch).reshape(-1, 1)

            Q_2 = critic.predict(s2_batch)

            R_batch = r_batch + GAMMA * Q_2 * (1-t_batch)
            
            predicted_Q, _ = critic.train(s_batch, np.reshape(R_batch, (-1, 1)), lr)

            td_batch = R_batch - predicted_Q

            actor.train(s_batch, a_batch, np.reshape(td_batch, (-1, 1)), lr)
                
        if terminal == False:
            var = var/MAX_STEP
            avg_var += (var/print_ep)
        if (r >= 100):
            ep_success[ep] = 1
        elif (r <= -100):
            ep_crash[ep] = 1
        
        if ((ep % print_ep) == (print_ep-1)) or (ep == 0):
            print('Episode: %d --- Ave R: %.1f  |  Success: %.2f  |  crash: %.2f  |  Max R: %.1f  |  var: %.2f  |  lr:%.2f*1e-4'\
                  %(ep, np.mean(ep_reward[last_ep:ep+1]), np.mean(ep_success[last_ep:ep+1]), \
                    np.mean(ep_crash[last_ep:ep+1]), np.amax(ep_reward[last_ep:ep+1]), var, 1e4*lr))
            last_ep = ep
            avg_var = 0
        if (ep % save_ep) == (save_ep-1):
            print(' ***** Saving...  at episode %d ******' %ep)
            saver = tf.train.Saver()
            filename = save_dir + 'Episod_' + str(ep) + '.chk'
            saver.save(sess, filename)
            with open(save_dir + 'rewards.txt', 'w') as f:
                for i in xrange(MAX_EPISODE):
                    f.write('%f\n' %ep_reward[i])
                    
        if (ep == 0):
            try:
                with open(save_dir + 'params.txt', 'w') as f:
                    f.write('Learning rate = %f\n' %LEARNING_RATE)
                    f.write('lr decay = %f\n' %decay_rate)
                    f.write('Batch size = %d\n' %MINIBATCH_SIZE)
                    f.write('actor hidden layers: %d\n' %info.act_hid_layers)
                    f.write('critic hidden layers: %d\n' %info.crit_hid_layers)
            except:
                os.mkdir(save_dir)  
                with open(save_dir + 'params.txt', 'w') as f:
                    f.write('Learning rate = %f\n' %LEARNING_RATE)
                    f.write('lr decay = %f\n' %decay_rate)
                    f.write('Batch size = %d\n' %MINIBATCH_SIZE)
                    f.write('actor hidden layers: %d\n' %info.act_hid_layers)
                    f.write('critic hidden layers: %d\n' %info.crit_hid_layers)
            
        if (ep % lr_decay_ep) == (lr_decay_ep-1):
            lr = decay_rate * lr

    
print('DONE') 
    

Instructions for updating:
Use `tf.global_variables_initializer` instead.


[2016-12-28 21:42:51,857] From <ipython-input-21-dd758e175a2f>:13 in <module>.: initialize_all_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Use `tf.global_variables_initializer` instead.


Episode: 0 --- Ave R: -428.3  |  Success: 0.00  |  crash: 1.00  |  Max R: -428.3  |  var: 0.25  |  lr:100.00*1e-4
Episode: 99 --- Ave R: -241.1  |  Success: 0.00  |  crash: 1.00  |  Max R: -15.2  |  var: 0.25  |  lr:100.00*1e-4
Episode: 199 --- Ave R: -221.5  |  Success: 0.00  |  crash: 1.00  |  Max R: -33.1  |  var: 0.25  |  lr:100.00*1e-4
Episode: 299 --- Ave R: -253.6  |  Success: 0.00  |  crash: 1.00  |  Max R: -37.9  |  var: 0.26  |  lr:100.00*1e-4
Episode: 399 --- Ave R: -244.8  |  Success: 0.00  |  crash: 0.99  |  Max R: 12.2  |  var: 0.26  |  lr:100.00*1e-4
Episode: 499 --- Ave R: -251.8  |  Success: 0.00  |  crash: 1.00  |  Max R: -43.6  |  var: 0.26  |  lr:100.00*1e-4
Episode: 599 --- Ave R: -257.3  |  Success: 0.00  |  crash: 0.99  |  Max R: 18.7  |  var: 0.27  |  lr:100.00*1e-4
Episode: 699 --- Ave R: -243.5  |  Success: 0.00  |  crash: 0.98  |  Max R: 64.5  |  var: 0.26  |  lr:100.00*1e-4
Episode: 799 --- Ave R: -247.6  |  Success: 0.00  |  crash: 1.00  |  Max R: -27.0  | 

[2016-12-28 21:44:26,005] Error encountered when serializing data_augmentation.
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'NoneType' object has no attribute 'name'


Type is unsupported, or the types of the items don't match field type in CollectionDef.
'NoneType' object has no attribute 'name'


...

Episode: 19099 --- Ave R: -171.4  |  Success: 0.00  |  crash: 1.00  |  Max R: -38.7  |  var: 0.39  |  lr:100.00*1e-4
Episode: 19199 --- Ave R: -164.6  |  Success: 0.00  |  crash: 1.00  |  Max R: -22.7  |  var: 0.49  |  lr:100.00*1e-4
Episode: 19299 --- Ave R: -170.0  |  Success: 0.00  |  crash: 1.00  |  Max R: -60.0  |  var: 0.28  |  lr:100.00*1e-4
Episode: 19399 --- Ave R: -170.5  |  Success: 0.00  |  crash: 1.00  |  Max R: -76.2  |  var: 0.33  |  lr:100.00*1e-4
Episode: 19499 --- Ave R: -165.1  |  Success: 0.00  |  crash: 1.00  |  Max R: -57.6  |  var: 0.47  |  lr:100.00*1e-4
Episode: 19599 --- Ave R: -158.8  |  Success: 0.00  |  crash: 1.00  |  Max R: -36.7  |  var: 0.38  |  lr:100.00*1e-4
Episode: 19699 --- Ave R: -158.8  |  Success: 0.00  |  crash: 1.00  |  Max R: -90.2  |  var: 0.46  |  lr:100.00*1e-4
Episode: 19799 --- Ave R: -152.9  |  Success: 0.00  |  crash: 1.00  |  Max R: -60.9  |  var: 0.33  |  lr:100.00*1e-4
Episode: 19899 --- Ave R: -153.7  |  Success: 0.00  |  crash: 1.

[2016-12-28 22:21:46,597] Error encountered when serializing data_augmentation.
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'NoneType' object has no attribute 'name'


Type is unsupported, or the types of the items don't match field type in CollectionDef.
'NoneType' object has no attribute 'name'


[2016-12-28 22:21:46,602] Error encountered when serializing data_preprocessing.
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'NoneType' object has no attribute 'name'


DONE


In [ ]:
EP = MAX_EPISODE - 1
with tf.Session(graph=graph) as sess:
    
    actor.set_up(sess)
    critic.set_up(sess)
    sess.run(tf.initialize_all_variables())
    saver = tf.train.Saver()
    filename = './saved_nets/Episod_' + str(EP) + '.chk'
    saver.restore(sess, filename)
    
    ep_reward = 0
    s = game.reset()
    for st in xrange(MAX_STEP):
        game.render()
        print(actor.give_policy(s.reshape(-1,info.dim_s)))
        A = actor.take_action(s.reshape(-1,info.dim_s))
        a = np.argmax(A)
        s2, r, terminal, report = game.step(a)
        s = s2
        ep_reward += (1.*r)
        if terminal:
            break
    print(r)
    print('reward:%f' %np.mean(ep_reward))

In [12]:
os.mkdir('./Results/a')


## 